# SSD: Single Shot MultiBox Detector

## 摘要  


该文章在既保证速度，又要保证精度的情况下，提出了 SSD 物体检测模型，与现在流行的检测模型一样，将检测过程整合到一个检测网络中。便于训练与优化，同时提高检测速度。SSD 将输出一系列离散化的 bounding boxes，这些 bounding boxes 是在不同层次（layers）上的 feature maps 上生成的，并且有着不同的 aspect ratio。在 prediction 阶段：要计算出每一个 default box 中的物体，其属于每个类别的可能性，即 score，得分。如对于 PASCAL VOC 数据集，总共有 20 类，那么得出每一个 bounding box 中物体属于这 20 个类别的每一种的可能性。同时，要对这些 bounding boxes 的 shape 进行微调，以使得其符合物体的 外接矩形。还有就是，为了处理相同物体的不同尺寸的情况，SSD 结合了不同分辨率的 feature maps 的 predictions。相对于那些需要 object proposals 的检测模型，本文的 SSD 方法完全取消了 proposals generation、pixel resampling 或者 feature resampling 这些阶段。这样使得 SSD 更容易去优化训练，也更容易地将检测模型融合进系统之中。在 PASCAL VOC、MS COCO、ILSVRC 数据集上的实验显示，SSD 在保证精度的同时，其速度要比用 region proposals 的方法要快很多。SSD 相比较于其他单结构模型（YOLO），SSD 取得更高的精度，即是是在输入图像较小的情况下。如输入 300×300 大小的 PASCAL VOC 2007 test 图像，在 Titan X 上，SSD 以 58 帧的速率，同时取得了 72.1% 的 mAP。
如果输入的图像是 500×500，SSD 则取得了 75.1% 的 mAP，比目前最 state-of-art 的 Faster R-CNN 要好很多。
  
SSD方法的贡献：  

提出了新的物体检测方法：SSD，比原先最快的 YOLO: You Only Look Once 方法，还要快，还要精确。保证速度的同时，检测效果 mAP 可以达到 region proposals 技术的方法（如 Faster R-CNN）等。  

SSD 方法的核心就是 predict object，以及其归属类别的 score；同时，在 feature map 上使用小的卷积核，去 predict 一系列 bounding boxes 的 box offsets。  

本文中为了得到高精度的检测结果，在不同层次的 feature maps 上去 predict object、box offsets，同时，还得到不同 aspect ratio predictions。  

本文的这些改进设计，能够在当输入分辨率较低的图像时，保证检测的精度。同时，这个整体 end-to-end 的设计，训练也变得简单。在检测速度、检测精度之间取得较好的 trade-off。  

本文提出的模型（model）在不同的数据集上，如 PASCAL VOC、MS COCO、ILSVRC，都进行了测试。在检测时间、检测精度上，均与目前物体检测领域 state-of-art 的检测方法进行了比较。

## 1 网络模型  

SSD 是一个前向传播 CNN 网络，产生一系列固定大小的 bounding boxes，以及每一个 box 中包含物体实例的可能性，之后进行非极大值抑制得到最终的 predictions。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/SSD.jpg?raw=true)

SSD 模型的最开始部分，本文称作 base network，是用于图像分类的标准架构。在 base network 之后，本文添加了额外辅助的网络结构：  

Multi-scale feature maps for detection   
在基础网络结构后，添加了额外的卷积层，这些卷积层的大小是逐层递减的，可以在多尺度下进行 predictions。  

Convolutional predictors for detection   
每一个添加的特征层（或者在基础网络结构中的特征层），可以使用一系列 convolutional filters，去产生一系列固定大小的 predictions。对于一个大小为 m×n，具有 p 通道的特征层，使用的 convolutional filters 就是 3×3×p 的 kernels。产生的 predictions，要么就是归属类别的一个得分，要么就是相对于 default box coordinate 的 shape offsets。  

default box 以及 feature map cell：  
feature map cell 就是将 feature map 切分成 5×5 大小的格子；  
而 default box 就是每一个格子上，一系列固定大小的 box  
如下图所示。在 5×5 大小的 feature map 上面，每个位置都生成3个不同大小的box，则该 feature map 对应的default box 共75个
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/SSD4.jpg?raw=true)

Default boxes and aspect ratios   
每一个 box 相对于与其对应的 feature map cell 的位置是固定的。在每一个 feature map cell 中，我们要 predict 得到的 box 与 default box 之间的 offsets，以及每一个 box 中包含物体的 score。   
因此，对于一个位置上的 k 个boxes 中的每一个 box，我们需要计算出 c 个类，每一个类的 score，还有这个 box 相对于它的默认 box 的 4 个偏移值（offsets）。于是，在 feature map 中的每一个 feature map cell 上，就需要有 (c+4)×k 个 filters。对于一张 m×n 大小的 feature map，即会产生 (c+4)×k×m×n 个输出结果。
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/SSD3.jpg?raw=true)

如图所示：对5×5大小的 feature map 有 75 个default box，则用（21+4）× 3 个filter 生成待预测的特征图之后，产生(21+4)×3×5×5 个输出结果

**如上图所示：SSD预测的目标就是以一张图中所有 default box (7308个)为窗口, 看其窗口是否存在物体, 如果有物体, 预测其类别以及位置，无物体则预测为背景**  

## 2 数据预处理  

本文对训练数据做了 data augmentation，用来增加训练数据，提高MAP。  
每一张训练图像，随机的进行如下几种选择：
1 使用原始图像进行训练  

2 zoom out 之后的图像进行训练  

3 翻转之后的图像进行训练  

4 随机的采样一个 patch 进行训练  

